#Setting

In [1]:
!pip install transformers==4.17 datasets accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
from datasets import load_dataset

dataset = load_dataset('klue', 'sts')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
        num_rows: 11668
    })
    validation: Dataset({
        features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
        num_rows: 519
    })
})

# Fine-tune a pretrained model

## Prepare a dataset

In [3]:
dataset['train'][0]

{'guid': 'klue-sts-v1_train_00000',
 'source': 'airbnb-rtt',
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'labels': {'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1}}

In [4]:
dataset = dataset.flatten()

In [5]:
dataset = dataset.rename_column('labels.real-label', 'label')

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")


def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Map:   0%|          | 0/11668 [00:00<?, ? examples/s]

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

In [7]:
dataset['train'][0]

{'guid': 'klue-sts-v1_train_00000',
 'source': 'airbnb-rtt',
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'labels.label': 3.7,
 'label': 3.714285714285714,
 'labels.binary-label': 1}

In [8]:
tokenized_datasets['train']

Dataset({
    features: ['guid', 'source', 'sentence1', 'sentence2', 'labels.label', 'label', 'labels.binary-label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 11668
})

In [9]:
tokenized_datasets['train'][0]['label']

3.714285714285714

In [10]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(500))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(500))

## Train

## Train with PyTorch Trainer

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=1)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

### Training hyperparameters

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "klue-bert-base-finetuned-sts",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

### Evaluate

### Trainer

Create a [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) object with your model, training arguments, training and test datasets, and evaluation function:

In [13]:
# datasets 라이브러리에서 제공하는 Evaluation metric의 리스트를 확인
from datasets import list_metrics, load_metric
metrics_list = list_metrics()
len(metrics_list)
print(', '.join(metric for metric in metrics_list))

accuracy, bertscore, bleu, bleurt, brier_score, cer, character, charcut_mt, chrf, code_eval, comet, competition_math, confusion_matrix, coval, cuad, exact_match, f1, frugalscore, glue, google_bleu, indic_glue, mae, mahalanobis, mape, mase, matthews_correlation, mauve, mean_iou, meteor, mse, nist_mt, pearsonr, perplexity, poseval, precision, r_squared, recall, rl_reliability, roc_auc, rouge, sacrebleu, sari, seqeval, smape, spearmanr, squad, squad_v2, super_glue, ter, trec_eval, wer, wiki_split, xnli, xtreme_s, AlhitawiMohammed22/CER_Hu-Evaluation-Metrics, BucketHeadP65/confusion_matrix, BucketHeadP65/roc_curve, CZLC/rouge_raw, DaliaCaRo/accents_unplugged_eval, DarrenChensformer/eval_keyphrase, DarrenChensformer/relation_extraction, DoctorSlimm/bangalore_score, DoctorSlimm/kaushiks_criteria, Drunper/metrica_tesi, Felipehonorato/eer, Fritz02/execution_accuracy, GMFTBY/dailydialog_evaluate, GMFTBY/dailydialogevaluate, He-Xingwei/sari_metric, Ikala-allen/relation_extraction, JP-SystemsX/nD

<ipython-input-13-05db9eda101a>:3: FutureWarning: list_metrics is deprecated and will be removed in the next major version of datasets. Use 'evaluate.list_evaluation_modules' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics_list = list_metrics()


In [14]:
# STS의 metric은 pearsonr, F1 score을 사용
metric_pearsonr = load_metric("pearsonr") # peason r

<ipython-input-14-a3ca2d3b6066>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_pearsonr = load_metric("pearsonr") # peason r
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for pearsonr contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/pearsonr/pearsonr.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [15]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    pr = metric_pearsonr.compute(predictions=predictions,
                                  references=labels)
    return pr

In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [17]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, guid, sentence2, source, labels.binary-label, labels.label. If sentence1, guid, sentence2, source, labels.binary-label, labels.label are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 500
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 189


Epoch,Training Loss,Validation Loss,Pearsonr
1,No log,2.086760,0.665503
2,No log,1.265686,0.733142
3,No log,1.231304,0.739421


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, guid, sentence2, source, labels.binary-label, labels.label. If sentence1, guid, sentence2, source, labels.binary-label, labels.label are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to klue-bert-base-finetuned-sts/checkpoint-63
Configuration saved in klue-bert-base-finetuned-sts/checkpoint-63/config.json
Model weights saved in klue-bert-base-finetuned-sts/checkpoint-63/pytorch_model.bin
tokenizer config file saved in klue-bert-base-finetuned-sts/checkpoint-63/tokenizer_config.json
Special tokens file saved in klue-bert-base-finetuned-sts/checkpoint-63/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.for

TrainOutput(global_step=189, training_loss=0.7006784267526455, metrics={'train_runtime': 206.1843, 'train_samples_per_second': 7.275, 'train_steps_per_second': 0.917, 'total_flos': 394663039488000.0, 'train_loss': 0.7006784267526455, 'epoch': 3.0})

In [18]:
print(small_train_dataset["sentence1"][0])
print(small_train_dataset["sentence2"][0])

4인은 멀티탭을 1개 준비하시면 편할것입니다.
멀티탭 1개를 준비하시면 4명이 편리하게 이용하실 수 있습니다.


In [19]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, guid, sentence2, source, labels.binary-label, labels.label. If sentence1, guid, sentence2, source, labels.binary-label, labels.label are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 1.2313036918640137,
 'eval_pearsonr': 0.7394212649350029,
 'eval_runtime': 16.5106,
 'eval_samples_per_second': 30.283,
 'eval_steps_per_second': 3.816,
 'epoch': 3.0}

In [23]:
# huggingface pipeline 라이브러리에 sts task에 대한 기능을 제공하지 않는다

from transformers import pipeline

text1 = '축구 농구 야구 배구'
text2 = "농구 야구 배구 축구"

model_checkpoint = "/content/klue-bert-base-finetuned-sts/checkpoint-189"
sts_pipeline = pipeline("zero-shot-classification", model=model_checkpoint)

sts_pipeline(text1, text2)

loading configuration file /content/klue-bert-base-finetuned-sts/checkpoint-189/config.json
Model config BertConfig {
  "_name_or_path": "/content/klue-bert-base-finetuned-sts/checkpoint-189",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading configuration file /content/klue-bert-base-finetuned-sts/checkpoint-189/config.json
Mod

{'sequence': '축구 농구 야구 배구', 'labels': ['농구 야구 배구 축구'], 'scores': [0.5]}